In [3]:
import nltk
nltk.download('punkt')
import joblib
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rissu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

C:\Users\rissu\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rissu\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rissu\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rissu\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [3]:
file=open('intents.json')

In [4]:
intents=json.load(file)

In [5]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [6]:
words=[]
classes=[]
documents=[]
ignore=['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
wordd=[]
for w in words:
    if w not in ignore:
        wordd.append(stemmer.stem(w.lower()))
        

In [8]:
wordd=sorted(list(set(wordd)))

In [9]:
len(documents),len(classes),len(wordd)

(31, 9, 57)

In [10]:
training=[]
output=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for w in wordd:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

In [11]:
# resetting underlying graph data
tf.reset_default_graph()

# building neural network

net=tflearn.input_data(shape=[None,len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net=tflearn.regression(net)

# defining model and setting up tensorboard
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

# start training
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00408 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.00408 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.00393 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.00393 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:C:\Users\rissu\A_DataScience\ML_Model\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
len(train_x[0])

57

In [13]:
len(train_y[0])

9

In [14]:
import pickle
pickle.dump({'words':wordd,
             'classes':classes,
             'train_x':train_x,
             'train_y':train_y},open('training_data','wb'))

In [15]:
data=pickle.load(open('training_data','rb'))

In [16]:
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [17]:
with open('intents.json') as json_data:
    intents=json.load(json_data)

In [18]:
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\rissu\A_DataScience\ML_Model\Chatbot\model.tflearn


In [19]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [20]:
def bag_of_words(sentence,words,show_details=False):
    bag=[0]*len(words)
    sentence=clean_up_sentence(sentence)
    for s in sentence:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("%s"%w)
    return np.array(bag)

In [66]:
bag_of_words('What can I do for you?',words,show_details=True)

what
can
do
for
you


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [22]:
ERROR_THRESOLD=0.30
def classify(sentence):
    results=model.predict([bag_of_words(sentence,words)])[0]
    results=[[i,r] for i,r in enumerate(results) if r>ERROR_THRESOLD]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list
        

In [74]:
classify('What are your hours of operation')

[('hours', 0.98636603)]

In [127]:
def response(sentence,userID='123',show_details=False):
    results=classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag']==results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)
response("Bye Bye")

See you later, thanks for visiting


In [157]:
response('What is menu for today')

Our speciality for today is Chicken Tikka
